# OCR and LDA on Lit Review Docs

In [319]:
import pdfminer as pdf
import os
import glob
import pickle as pkl
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim
import unicodedata
import re, string
from gensim import corpora, models
import gensim

In [6]:
# Get a blob of the pdf filenames, and turn them into a list
files = glob.glob('/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/*.pdf')

In [9]:
files[0:5]

['/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/AutomatedRecHealthyPersMeals.pdf',
 '/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/Biologeek.pdf',
 '/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/BuonApetitoRestMealRec.pdf',
 '/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/ChallengesNutrRecSys.pdf',
 '/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/ClusteringEliminateSpam.pdf']

In [16]:
text_files = glob.glob('/Users/sararogis/Dropbox/FoodRecommender/MealRec_LitReview/LitRev_Text/*.txt')

In [357]:
for x in files:
    dest_loc = x.replace('.pdf','.txt').replace('MealRec_LitReview/','MealRec_LitReview/LitRev_Text/')
    ocr_cmd = 'pdf2txt.py -o ' + dest_loc + ' -t text ' + x
    os.system(ocr_cmd)

In [ ]:
text_files[0:5]

In [20]:
# Ok, now pull these text files in and lets do some lda

In [359]:
all_texts = []

In [360]:
for textpath in text_files:
    # Set an open string to place document text into
    all_doc_string = ''
    
    # Open the file and save contents to variable 'file_text'
    file_text = open(textpath, 'rb')
    
    # Read the lines of the tester into a variable. 
    file_lines = file_text.readlines()
    
    # Iterate through the lines, and append them to the string to make one big text blob
    for line in file_lines:
        all_doc_string = all_doc_string + line
        
    # Append the new string to the list of all files
    all_texts.append(all_doc_string)
    
    # String will reset next; close the file and flush the file buffer
    file_text.flush()
    file_text.close()

In [ ]:
all_texts

In [362]:
len(all_texts)

32

In [363]:
len(text_files)

32

In [364]:
# Hmmm. One off, but what the heck. Save it. 
pkl.dump(all_texts, open('all_texts_mealrec_litreview.pkl','wb'))

In [55]:
#####

In [365]:
df = pd.DataFrame(all_texts,columns=['orig_text'])

In [366]:
def preprocess_text(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import LineTokenizer
    from nltk.tokenize import WhitespaceTokenizer
    from nltk.stem.porter import PorterStemmer
    
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    
    sw = list(stopwords.words())
    extra_stops = ['R', '', ' ', 'abstract', 'keywords', 'introduction', 'figure','morgan', 'harvey',
                   'david','elsweiler','northumbria','university','newcastle','united','kingdom','university',
                   'regensburg','germany', 'h', 'k', 'f', 'b', 'user', 'g', 'use']
    for word in extra_stops:
        sw.append(word)
    
    # Step 1 - Clean up unicode
    clean_string = ''
    doc = []
    #for x in text:
    #    if ord(x) <= 128:
    #        clean_string += x
    #clean_string = unicodedata.normalize('NFKD', clean_string.encode('utf-8', 'replace')).encode('ascii','replace')
            
    # Tokenize each line to get rid of the line carriages
    lines = LineTokenizer().tokenize(text.lower())
    
    clean_lines = []
    
    for line in lines:
        if line.startswith('e-mail') or line.startswith('doi') or line.startswith('For all other uses, contact') or line.find(' acm. isbn ') > 0:
            pass
        else:
            line_str = ''
            for char in line:
                #if ord(char) <= 127:
                if (char in string.ascii_letters) or char == ' ':
                    line_str += char
        
        # Clean up other undesirable characters
            if line_str != ' ' and line_str.rstrip().lstrip() not in sw:
                clean_lines.append(line_str)
    
    # Tokenize the lines
    for clean_line in clean_lines:
        tokens = WhitespaceTokenizer().tokenize(clean_line)
        stopped_tokens = [i for i in tokens if not i in sw]
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        [doc.append(i) for i in stemmed_tokens]
        
    
    return doc

In [367]:
df.insert(df.shape[1], 'clean_text', df.orig_text.apply(preprocess_text))

In [369]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(df.clean_text)

In [370]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in df.clean_text]

In [371]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)

In [372]:
ldamodel4 = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word = dictionary, passes=20)

In [373]:
ldamodel3 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [374]:
print(ldamodel.print_topics(num_topics=10, num_words=4))

[(0, u'0.016*food + 0.013*algorithm + 0.012*cid + 0.009*r'), (1, u'0.000*food + 0.000*recip + 0.000*system + 0.000*recommend'), (2, u'0.028*recip + 0.023*menu + 0.015*recommend + 0.013*food'), (3, u'0.040*recip + 0.019*ingredi + 0.009*similar + 0.008*use'), (4, u'0.000*food + 0.000*recip + 0.000*use + 0.000*system'), (5, u'0.000*food + 0.000*recommend + 0.000*recip + 0.000*system'), (6, u'0.000*recip + 0.000*food + 0.000*system + 0.000*ingredi'), (7, u'0.048*food + 0.014*practic + 0.011*particip + 0.011*wast'), (8, u'0.017*system + 0.013*user + 0.012*product + 0.010*recip'), (9, u'0.024*recip + 0.022*recommend + 0.018*system + 0.016*user')]


In [375]:
import pyLDAvis

In [376]:
pyLDAvis.enable_notebook()

In [377]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      29.504711        1       1 -0.106822 -0.173555
9      27.416633        1       2 -0.168737  0.027879
3      13.685156        1       3 -0.090105  0.050671
8      11.325331        1       4 -0.130220 -0.122096
2       9.346031        1       5 -0.140154  0.090575
0       8.708459        1       6 -0.094952  0.128187
5       0.003420        1       7  0.181959 -0.000537
4       0.003420        1       8  0.182305 -0.000579
6       0.003420        1       9  0.183362 -0.000277
1       0.003420        1      10  0.183365 -0.000266, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
1248  Default  1565.000000      recip  1565.000000  30.0000  30.0000
300   Default   238.000000       menu   238.000000  29.0000  29.0000
3434  Default  1945.000000       food  1945.000000  28.0000  28.0000
7283  Default   402.000000    practic   402.000000  27.0000  27.0000
4185  Default   595.000000    ingredi   595.000000  26.0000  26.0000
4460  Default   902.000000  recommend   902.000000  25.0000  25.0000
216   Default   292.000000       wast   292.000000  24.0000  24.0000
1873  Default   895.000000     system   895.000000  23.0000  23.0000
1621  Default   371.000000  algorithm   371.000000  22.0000  22.0000
2435  Default   581.000000       rate   581.000000  21.0000  21.0000
5238  Default   276.000000       shop   276.000000  20.0000  20.0000
2845  Default   208.000000    product   208.000000  19.0000  19.0000
3607  Default   415.000000   particip   415.000000  18.0000  18.0000
1590  Default   183.000000        cid   183.000000  17.0000  17.0000
4901  Default   414.000000     featur   414.000000  16.0000  16.0000
4582  Default   824.000000       user   824.000000  15.0000  15.0000
2738  Default   118.000000      graph   118.000000  14.0000  14.0000
3804  Default   295.000000    similar   295.000000  13.0000  13.0000
620   Default   355.000000     prefer   355.000000  12.0000  12.0000
3531  Default   114.000000    groceri   114.000000  11.0000  11.0000
7382  Default   212.000000          r   212.000000  10.0000  10.0000
198   Default   110.000000      navig   110.000000   9.0000   9.0000
3921  Default   134.000000      fridg   134.000000   8.0000   8.0000
1162  Default   282.000000     social   282.000000   7.0000   7.0000
101   Default   142.000000  household   142.000000   6.0000   6.0000
4267  Default   157.000000       home   157.000000   5.0000   5.0000
3642  Default    90.000000       imag    90.000000   4.0000   4.0000
1343  Default   300.000000       plan   300.000000   3.0000   3.0000
1385  Default   315.000000        set   315.000000   2.0000   2.0000
304   Default    97.000000       busi    97.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
3434  Topic10     0.000497       food  1945.074469  -4.8957  -8.7684
1248  Topic10     0.000486      recip  1565.882060  -4.7017  -8.7912
4460  Topic10     0.000454  recommend   902.943697  -4.2194  -8.8594
4582  Topic10     0.000437       user   824.684741  -4.1673  -8.8980
4901  Topic10     0.000422     featur   414.842097  -3.5155  -8.9333
2435  Topic10     0.000424       rate   581.066894  -3.8464  -8.9272
7291  Topic10     0.000423     inform   513.501629  -3.7264  -8.9308
4185  Topic10     0.000420    ingredi   595.032866  -3.8801  -8.9372
3804  Topic10     0.000409    similar   295.426228  -3.2075  -8.9648
8308  Topic10     0.000415        use   661.088914  -3.9983  -8.9501
765   Topic10     0.000410        one   367.100892  -3.4229  -8.9629
3607  Topic10     0.000405   particip   415.519210  -3.5588  -8.9750
2782  Topic10     0.000402      gener   266.748068  -3.1219  -8.9813
7283  Topic10     0.000403    practic   402.610950  -3.5301  -8.9778
7178  Topic10     0.000399       cook   374.983324  -3.4695  -8.98

In [378]:
pyLDAvis.gensim.prepare(ldamodel3, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      37.912012        1       1  0.136382  0.005009
2      32.402903        1       2 -0.059993 -0.064994
0      29.685085        1       3 -0.076389  0.059986, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
1248  Default  1498.000000      recip  1498.000000  30.0000  30.0000
4185  Default   572.000000    ingredi   572.000000  29.0000  29.0000
4460  Default   869.000000  recommend   869.000000  28.0000  28.0000
3434  Default  1875.000000       food  1875.000000  27.0000  27.0000
7283  Default   384.000000    practic   384.000000  26.0000  26.0000
2435  Default   560.000000       rate   560.000000  25.0000  25.0000
216   Default   278.000000       wast   278.000000  24.0000  24.0000
5238  Default   266.000000       shop   266.000000  23.0000  23.0000
3607  Default   397.000000   particip   397.000000  22.0000  22.0000
1621  Default   365.000000  algorithm   365.000000  21.0000  21.0000
300   Default   232.000000       menu   232.000000  20.0000  20.0000
1873  Default   869.000000     system   869.000000  19.0000  19.0000
5053  Default   267.000000    predict   267.000000  18.0000  18.0000
4901  Default   403.000000     featur   403.000000  17.0000  17.0000
101   Default   135.000000  household   135.000000  16.0000  16.0000
3921  Default   128.000000      fridg   128.000000  15.0000  15.0000
4267  Default   150.000000       home   150.000000  14.0000  14.0000
620   Default   344.000000     prefer   344.000000  13.0000  13.0000
2389  Default   163.000000    network   163.000000  12.0000  12.0000
5986  Default    95.000000    patient    95.000000  11.0000  11.0000
2845  Default   203.000000    product   203.000000  10.0000  10.0000
3531  Default   111.000000    groceri   111.000000   9.0000   9.0000
1590  Default   182.000000        cid   182.000000   8.0000   8.0000
4582  Default   797.000000       user   797.000000   7.0000   7.0000
3804  Default   288.000000    similar   288.000000   6.0000   6.0000
3464  Default   104.000000    sustain   104.000000   5.0000   5.0000
1173  Default   221.000000     select   221.000000   4.0000   4.0000
1198  Default   140.000000      learn   140.000000   3.0000   3.0000
7244  Default   191.000000      share   191.000000   2.0000   2.0000
2797  Default   136.000000      mobil   136.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
1590   Topic3   116.335648        cid   182.626755   0.7636  -5.4747
453    Topic3    98.113598     measur   156.348428   0.7486  -5.6451
4566   Topic3    77.992371    content   119.021315   0.7918  -5.8746
2738   Topic3    76.394208      graph   117.468683   0.7843  -5.8953
5231   Topic3    47.387690       pair    65.724372   0.8874  -6.3728
5744   Topic3    41.331526  construct    55.671856   0.9167  -6.5096
3804   Topic3   154.295558    similar   288.688616   0.5880  -5.1923
1385   Topic3   149.667329        set   307.295368   0.4951  -5.2228
4460   Topic3   303.947331  recommend   869.948741   0.1629  -4.5143
5603   Topic3   106.617930        two   219.554988   0.4922  -5.5619
7291   Topic3   186.738116     inform   494.933113   0.2398  -5.0015
6183   Topic3    73.382947      onlin   135.766404   0.5993  -5.9355
8308   Topic3   201.452628        use   639.866665   0.0588  -4.9256
673    Topic3    98.696667       data   239.673706   0.3273  -5.6391
1621   Topic3   126.615187  algorithm   365.635648   0.1540  -5.3900
4582   Topic3   199.076254       user   797.078290  -0.1727  -4.9375
7178   Topic3   121.607153       cook   359.243557   0.1313  -5.4304
6946   Topic3    81.399526   approach   182.828855   0.4053  -5.8318
3475   Topic3    98.825026       base   266.463716   0.2226  -5.6378
765    Topic3   114.680630        one   355.089897   0.0843  -5.4890
4901   Topic3   118.036182    